In [6]:
import praw
import json
from datetime import datetime

reddit = praw.Reddit(
    client_id='B-qrtXmBJd2BgAAwmKozQg',
    client_secret='iVwh4ZpT35sQMBvye7nKaz8839LDPQ',
    user_agent='YOUR_USER_AGENT'
)

subreddit_name = 'IndianSkincareAddicts'
subreddit = reddit.subreddit(subreddit_name)
search_query = 'Love Beauty and Planet'
search_results = subreddit.search(search_query, limit=30)

parsed_posts = []

for post in search_results:
    # Convert UTC to a readable date-time format
    created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    
    # Construct the full URL
    full_permalink = f"https://www.reddit.com{post.permalink}"
    
    # Check if the post has media or images
    media_url = None
    if hasattr(post, 'media') and post.media:
        media_url = post.media.get('reddit_video', {}).get('fallback_url')
    elif hasattr(post, 'preview') and post.preview:
        media_url = post.preview.get('images', [{}])[0].get('source', {}).get('url')
    
    # Append post data to the list
    parsed_posts.append({
        'post_title': post.title,
        'post_url': full_permalink,
        'score': post.score,
        'author': str(post.author),
        'created_date': created_date,
        'num_comments': post.num_comments,
        'selftext': post.selftext,
        'media_url': media_url  # Include media URL if available
    })

# Save to JSON
output_file_path = 'product_mentions.json'
with open(output_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(parsed_posts, json_file, ensure_ascii=False, indent=2)

print(f"Data has been saved to {output_file_path}")

/var/folders/4c/6f4zjqbs5tv0rl2p0nbskkp40000gn/T/ipykernel_28124/2388658200.py:20: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')


Data has been saved to product_mentions.json
